In [22]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [2]:
### load the trained model, scaler picke, onehot

model = load_model('model.h5')

## load the encoders and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [20]:
# Create example input data as a dictionary
input_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender': 'Female', 
    'Age': 42,
    'Tenure': 2,
    'Balance': 0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88
}


In [26]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df= pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\python_ai_projects\ann_classification\ann_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [28]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0,2,1,1,101348.88


In [ ]:
# encode gender

In [31]:
## encode categoricalvar
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0,2,1,1,101348.88


In [32]:
## conbine ooenhot encoded col with input data
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df], axis=1)  # resent index means same index as prev
input_df
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0,2,1,1,101348.88,1.0,0.0,0.0


In [33]:
## scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
         0.80843615,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [35]:
## prediction churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 42ms/step


array([[0.01034621]], dtype=float32)

In [36]:
prediction_probaility = prediction[0][0]
prediction_probaility

0.010346211

In [37]:
if prediction_probaility > 0.5 : print("The customer is likely to churn ")
else: print("the customer is unlikely to churn")

the customer is unlikely to churn
